In [1]:
import sys
import numpy as np
import qutip as qt
import pickle

sys.path.append('../../')
from Pulses import *
from Noises import Noise
from Plots import plot_populations, plot_pulses
from joblib import Parallel, delayed, parallel
from tqdm.auto import tqdm
import contextlib
from Pulses import CTAP_pulses, STA_pulses
from Plots import plot_best_episode

%load_ext autoreload
%autoreload
from QEnvs.QEnvWave import QEnvWave

2023-06-09 23:08:51.406471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 23:08:52.159354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_qubits = 3
time_max = 50
num_steps = 100
percentage_points = 50
omega_max = 1
initial_state = qt.basis(3, 0)
target_state = qt.basis(3, 2)
noise_type = "gaussian"
noise_max_percent = 0.1

pulses_ctap = CTAP_pulses(num_qubits, time_max, num_steps, time_max/6, time_max/6, omega_max)
pulses_sta = STA_pulses(num_qubits, time_max, num_steps, omega_max)

pulses = np.array([pulses_ctap, pulses_sta, np.load("Data/CRAB_50.npy"), np.load("Data/GRAPE_50.npy")])

cases = ["CTAP", "STA", "CRAB", "GRAPE"]

In [3]:
def dummy_cost_function(env):
    return 0.0

In [4]:
noise = Noise(noise_type, 0.)
env = QEnvWave(num_qubits=num_qubits,
                   time_max=time_max,
                   num_steps=num_steps,
                   cost_function=dummy_cost_function,
                   omega_max=omega_max,
                   noise=noise)

In [5]:
def calculate_populations(env, pulse):
    times = env.times
    populations = []
    for state in env.run_qevolution(pulse):
        populations.append(np.abs(state))
    return times, np.array(populations).reshape((num_steps + 1, num_qubits))

In [6]:
def standard_deviation_fidelities(env, pulse, number_of_runs):
    fidelities = np.zeros(number_of_runs)
    for i in range(number_of_runs):
        fidelities[i] = env._quantum2rlstate(env.run_qevolution(pulse)).reshape((num_steps + 1, num_qubits))[-1,-1]
    return np.std(fidelities)

def number_of_runs(env, pulse, omega_percentage=0.005, number_of_runs=10):
    omeag_last = 0
    omega = standard_deviation_fidelities(env, pulse, number_of_runs)
    while omega - omeag_last > omega_percentage:
        number_of_runs *= 2
        omeag_last = omega
        omega = standard_deviation_fidelities(env, pulse, number_of_runs)
    return number_of_runs

In [7]:
def noise_percentage_effect(env, pulse, noise_max_percent):
    noise_percentages = np.linspace(0, noise_max_percent, percentage_points)
    env.noise.percentage = noise_max_percent
    print("Calculating number of runs")
    num_runs = number_of_runs(env, pulse, omega_percentage=0.001)
    results = np.zeros((len(noise_percentages), 2))
    print("Starting probing noises")
    for i, noise_percentage in enumerate(noise_percentages):
        env.noise.percentage = noise_percentage
        fidelities = np.zeros(num_runs)
        for j in range(num_runs):
            fidelities[j] = env._quantum2rlstate(env.run_qevolution(pulse)).reshape((num_steps + 1, num_qubits))[-1,-1]
        results[i, 0] = np.mean(fidelities)
        results[i, 1] = np.std(fidelities)
    return results, noise_percentages

In [8]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar."""
    class TqdmBatchCompletionCallback(parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = parallel.BatchCompletionCallBack
    parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [9]:
def calculations(env, label, pulse, noise_max_percent):
    times, populations = calculate_populations(env, pulse)
    noise_results, noise_percentages = noise_percentage_effect(env, pulse, noise_max_percent)
    return np.array([label, times, pulse, populations, noise_percentages, noise_results])

In [10]:
with tqdm_joblib(tqdm(range(len(pulses)), desc="Computing noisy transfer")) as pbar:
    results = Parallel(n_jobs=-1)(delayed(calculations)(env, cases[i], pulse, noise_max_percent) for i, pulse in enumerate(pulses))

Computing noisy transfer:   0%|          | 0/4 [00:00<?, ?it/s]

2023-06-09 23:08:56.276850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 23:08:56.330827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 23:08:56.423266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-09 23:08:56.443513: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Calculating number of runs
Calculating number of runs
Calculating number of runs
Calculating number of runs
Starting probing noises
Starting probing noises
Starting probing noises
Starting probing noises


/tmp/ipykernel_58924/3707702147.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/tmp/ipykernel_58924/3707702147.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/tmp/ipykernel_58924/3707702147.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/tmp/ipykernel_58924/3707702147.py:4: VisibleDeprecationWarning: Creating an ndarray fro

In [11]:
# Save results
with open("Data/sota_protocols_comparison_noise.pkl", "wb") as f:
    pickle.dump(results, f)